# Second Approch

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scp
from sympy import Symbol
import copy as cp
import seaborn as sns
from scipy.linalg import cholesky
from math import sqrt

In this approch, the objectif is to use the modeling equation (1) of $H$, the water level of the watercourse, in order to minimize the overflow height, $S$, that is related to the physical variables and the height of the dike by the equation (2).

$H = {\left(\frac{Q}{K_s \sqrt(\frac{Z_m - Z_v}{L}) B}\right)}^{\frac{3}{5}} \ (1)$

$S = Z_c - Z_d = Zv + H - \mathcal{H}_d - Z_m \ (2) $

By the formulation of the problem, the major sources of uncertainty are considered to be the variables $Q$, $Z_m$, $Z_v$ and $K_s$. In this sense these variables are considered to be random variables with the following distributions :

$Q \sim Gumbel(1013, 558)$

$Z_m \sim Triangular(55, halfrange = 1)$

$Z_v \sim Triangular(50, halfrange = 1)$

$K_s \sim \mathcal{N}(30, 7.5)$

The other variables, $B$, $L$ and $Z_b$, are considered to be deterministics with values 300, 5000 and 55,5 respectively.

For the development that follows, it will be considered the 3 different scenarios:

1) Random variables without correlation

2) Correlation $K_s$-$Z_v$ of -0.6 and $Z_v$-$Z_m$ of 0.6

3) Correlation $K_s$-$Z_v$ of -0.3, $Z_v$-$Z_m$ of 0.66 and $K_s$-$Z_m$ of -0.5

-----

1) Generating realisations of the random variables and defining constants for the 3 scenarios

In [38]:
N = 500

#first scenario
q_realisations = np.random.gumbel(1031,558,N)
zm_realisations = np.random.triangular(54,55,56,N)
zv_realisations = np.random.triangular(49,50,51,N)
ks_realisations = np.random.normal(30,7.5,N)

#second scenario
cov_matrix2 = np.array([
        [  1, -0.6, 0],
        [ -0.6,  1,  0.6],
        [ 0,  0.6,  1]
    ])

c2 = cholesky(cov_matrix2, lower=True)

realisations_2 =[]
for i in range(N):
    uncorrelated_var_i = [ks_realisations[i],zv_realisations[i],zm_realisations[i]]
    correlated_var_i = np.dot(c2,uncorrelated_var_i)
    realisations_2.append([q_realisations[i],correlated_var_i[0],correlated_var_i[1],correlated_var_i[2]])

#third scenario
cov_matrix3 = np.array([
        [  1, -0.3, -0.5],
        [ -0.3,  1,  0.66],
        [ -0.5,  0.66,  1]
    ])

c3 = cholesky(cov_matrix3, lower=True)

print(c3)

realisations_3 =[]
for i in range(N):
    uncorrelated_var_i = [ks_realisations[i],zv_realisations[i],zm_realisations[i]]
    correlated_var_i = np.dot(c3,uncorrelated_var_i)
    realisations_3.append([q_realisations[i],correlated_var_i[0],correlated_var_i[1],correlated_var_i[2]])


B=300
L=5000
Zb=55.5

[[ 1.          0.          0.        ]
 [-0.3         0.9539392   0.        ]
 [-0.5         0.53462527  0.6813045 ]]


2) Generating H in the diferent scenarios

In [35]:
#first scenario

H_1 = np.zeros(N)
for i in range(N):
    Q_1 = np.power(abs(q_realisations[i]),3/5)*np.sign(q_realisations[i])
    Div_1 = np.power(1/(ks_realisations[i]*sqrt((zm_realisations[i]-zv_realisations[i])/L)*B),3/5)
    H_1[i] = Q_1*Div_1

#second scenario

H_2 = np.zeros(N)
for i in range(N):
    Q_2 = np.power(abs(realisations_2[i][0]),3/5)*np.sign(realisations_2[i][0])
    Div_2 = np.power(1/(realisations_2[i][1]*sqrt((realisations_2[i][3]-realisations_2[i][2])/L)*B),3/5)
    H_2[i]=Q_2*Div_2


#third scenario

H_3 = np.zeros(N)
for i in range(N):
    Q_3 = np.power(abs(realisations_3[i][0]),3/5)*np.sign(realisations_3[i][0])
    Div_3 = np.power(1/(realisations_3[i][1]*sqrt((realisations_3[i][3]-realisations_3[i][2])/L)*B),3/5)
    H_3[i]=Q_3*Div_3
